In [1]:
import pandas as pd
import numpy as np

# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier


from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image


import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="ticks", palette="pastel")

In [5]:
### RBL inputs into general FOOOF approach below:
dirs = {'nk':       '/Volumes/DBS Pain 3/nk_device_data/processed/fooof_specs (30s_prior_to_survey, updated channels)/',
        'rcs':      '/Volumes/DBS Pain 3/rcs_device_data/processed/spectra_per_sess/',
        'rcs_sub':  '/stage1_only (pp work up)/30s_pre_survey/',
        'both_sub': '/exported_fooof_data/'
        }

In [6]:
i        =0
pts      = ['RCS02', 'RCS04', 'RCS05', 'RCS06', 'RCS07']
pt_sides = ['RCS02R', 'RCS04R', 'RCS04L', 'RCS05R', 'RCS05L', 'RCS06R','RCS06L','RCS07L', 'RCS07R']

In [4]:
nk_bands = pd.read_excel(dirs['nk'] + 'ALL_bands.xlsx')
rcs_bands = pd.read_excel(dirs['rcs'] + 'ALL_bands.xlsx')

In [5]:
melt_list     = []
nk_bands_hemi = list(nk_bands.groupby(by='pt_hemi'))

for i in range(len(pt_sides)):
  
    hemi_bands                   = pd.melt(nk_bands_hemi[i][1][['theta_pwr', 'alpha_pwr', 'b_low_pwr','b_high_pwr']])
    hemi_bands.loc[:, 'pt_hemi'] = nk_bands_hemi[i][0]
    hemi_bands.loc[:, 'stage']   = 'inpatient'
    melt_list.append(hemi_bands)

rcs_bands_hemi = list(rcs_bands.groupby(by='pt_hemi'))

for i in range(len(pt_sides)):
  
    hemi_bands                   = pd.melt(rcs_bands_hemi[i][1][['theta_pwr', 'alpha_pwr', 'b_low_pwr','b_high_pwr']])
    hemi_bands.loc[:, 'pt_hemi'] = rcs_bands_hemi[i][0]
    hemi_bands.loc[:, 'stage']   = 'ambulatory'
    melt_list.append(hemi_bands)

    
bands_melt = pd.concat(melt_list, ignore_index=True).rename(columns={"variable": "power_band", "value": "peak_pwr"})

bands_melt

,power_band,peak_pwr,pt_hemi,stage
0,theta_pwr,NaN,RCS02R,inpatient
1,theta_pwr,NaN,RCS02R,inpatient
2,theta_pwr,NaN,RCS02R,inpatient
3,theta_pwr,NaN,RCS02R,inpatient
4,theta_pwr,NaN,RCS02R,inpatient
...,...,...,...,...
9943,b_high_pwr,0.169596,RCS07R,ambulatory
9944,b_high_pwr,0.178597,RCS07R,ambulatory
9945,b_high_pwr,0.220496,RCS07R,ambulatory
9946,b_high_pwr,0.218387,RCS07R,ambulatory


In [ ]:
fig = plt.figure(figsize=(16, 10))

fig.patch.set_facecolor('white')

plt.subplots_adjust(top = .95, hspace = 0.25, wspace = 0.25)

for i in range(len(pt_sides)):
    
    ax = plt.subplot(3, 3, i+1)
    plt.title(pt_sides[i], fontsize = 12)
    
    hemi_bands = bands_melt.loc[bands_melt.pt_hemi == pt_sides[i], :]



    sns.boxplot(x    = "power_band",
                y    = "peak_pwr",
                hue  = "stage",
                data = hemi_bands)
    
    ax.set_xlabel("") 
    ax.set_ylabel("Periodic Power (db)")
    
    plt.ylim([0,2])
    if i>0:
        ax.legend([],[], frameon=False)

NameError: name 'plt' is not defined

In [6]:
features = ['theta_freq', 'alpha_freq', 'b_low_freq','b_high_freq',
            'theta_pwr', 'alpha_pwr', 'b_low_pwr','b_high_pwr',
            'theta_width', 'alpha_width', 'b_low_width','b_high_width'];

X        = nk_bands[features].values
i_nan    = np.isnan(X)
X[i_nan] = 0

nk_bands[features]= X

In [11]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
                                                    nk_bands[features],
                                                    nk_bands['pt_hemi'], 
                                                    test_size=0.1,
                                                    random_state=42
                                                    )

In [12]:
# Logistic Regression
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)

accuracy_log_reg = accuracy_score(y_test, y_pred_log_reg)


print("Logistic Regression Accuracy:", accuracy_log_reg)

Logistic Regression Accuracy: 0.5443037974683544


In [13]:
his_boost = HistGradientBoostingClassifier(max_iter=100).fit(X_train, y_train)

y_pred_hist   = his_boost.predict(X_test)
accuracy_hist = accuracy_score(y_test, y_pred_hist)
print("HistGradientBoostingClassifier Accuracy:", accuracy_hist)

HistGradientBoostingClassifier Accuracy: 0.5759493670886076


In [14]:
# Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100)
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf     = rf_classifier.predict(X_test)
# Calculate accuracy scores
accuracy_rf = accuracy_score(y_test, y_pred_rf)

print("Random Forest Accuracy:", accuracy_rf)

Random Forest Accuracy: 0.5886075949367089
